In [1]:
import torch; print(torch.__version__)

0.4.1.post2


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import pickle

import multiprocessing

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
n_cpus = multiprocessing.cpu_count()

In [4]:
path = "../data/"
with open(f'{path}tfidf_matrix.pkl', 'rb') as fh:
    tfidf = pickle.load(fh)

In [5]:
tfidf = pd.DataFrame(tfidf.toarray()); tfidf.head()

,0,1,2,3,4,5,6,7,8,9,...,2785,2786,2787,2788,2789,2790,2791,2792,2793,2794
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
n_movies, n_tokens = tfidf.shape; n_movies, n_tokens

(26744, 2795)

In [7]:
maxVal = tfidf.values.max(); minVal = tfidf.values.min()
maxVal, minVal

(1.0, 0.0)

In [8]:
class AETrainingData(Dataset):
    '''
        Format the training dataset to be input into the auto encoder.
        Takes in dataframe and converts it to a PyTorch Tensor
    '''
    
    def __init__(self, x_train):
        self.x = x_train
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        '''
            Returns a example from the data set as a pytorch tensor.
        '''
        # Get example/target pair at idx as numpy arrays
        x, y = self.x.iloc[idx].values, self.x.iloc[idx].values

        # Convert to torch tensor
        x = torch.from_numpy(x).type(torch.FloatTensor)
        y = torch.from_numpy(y).type(torch.FloatTensor)
        
        # Return pair        
        return {'input': x, 'target': y}

In [28]:
class encoder(nn.Module):
    def __init__(self):
        super(encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(n_tokens, 1000),
            nn.ReLU(True),
            nn.Linear(1000, 100),
            nn.ReLU(True))

    def forward(self, x):
        x = self.encoder(x)
        return x
    
    
class decoder(nn.Module):
    def __init__(self):
        super(decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.Linear(100, 1000),
            nn.ReLU(True),
            nn.Linear(1000, n_tokens),
            nn.Sigmoid())

    def forward(self, x):
        x = self.decoder(x)
        return x

In [10]:
def train_ae(input_tensor, target_tensor, encoder, decoder,
          encoder_optimizer, decoder_optimizer, criterion):
    
    # clear the gradients in the optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    # Forward pass through 
    
    encoded_representation = encoder(input_tensor)
    reconstruction = decoder(encoded_representation)
    
    # Compute the loss
    loss = criterion(reconstruction, target_tensor)
    
    # Compute the gradients
    loss.backward()
    
    # Step the optimizers to update the model weights
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    # Return the loss value to track training progress
    return loss.item()

In [24]:
def trainIters(encoder, decoder, dataloader, epochs, print_every_n_batches=100, learning_rate=0.0001):
    
    # keep track of losses
    plot_losses = []

    # Initialize Encoder Optimizer
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate, weight_decay=1e-8)
    
    # Initialize Decoder Optimizer
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate, weight_decay=1e-8)

    # Specify loss function
    criterion = nn.MSELoss(reduction='elementwise_mean')
    
    # Cycle through epochs
    for epoch in range(epochs):

        print(f'Epoch {epoch + 1}/{epochs}')
        # Cycle through batches
        for i, batch in enumerate(dataloader):
            
            input_tensor = batch['input'].cuda()
            target_tensor = batch['target'].cuda()
            

            loss = train_ae(input_tensor, target_tensor, encoder, decoder,
                         encoder_optimizer, decoder_optimizer, criterion)
            

            if i % print_every_n_batches == 0 and i != 0:
                print(loss)
                plot_losses.append(loss)
    return plot_losses

In [25]:
dataset = AETrainingData(tfidf)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=8)

In [29]:
encoder = encoder().cuda()
decoder = decoder().cuda()

In [31]:
trainIters(encoder, decoder, dataloader, 30)

Epoch 1/30
0.00025159408687613904
0.0002567078627180308
0.0002347471017856151
0.0002623775217216462
Epoch 2/30
0.0002566072216723114
0.00025404829648323357
0.00025616673519834876
0.00027566897915676236
Epoch 3/30
0.00022701302077621222
0.00026205735048279166
0.00026155426166951656
0.0002506707387510687
Epoch 4/30
0.0002485751756466925
0.000243923015659675
0.00021555279090534896
0.00024902125005610287
Epoch 5/30
0.000250117271207273
0.00026644422905519605
0.00021542387548834085
0.00020635980763472617
Epoch 6/30
0.0002318520419066772
0.00024445666349492967
0.0002500811533536762
0.0002262728230562061
Epoch 7/30
0.00022470254043582827
0.0002388863795204088
0.00023566283925902098
0.0002182702737627551
Epoch 8/30
0.00025422341423109174
0.0002464283024892211
0.00022828884539194405
0.00024847075110301375
Epoch 9/30
0.00020504763233475387
0.00023537101515103132
0.0001939756766660139
0.00023538354435004294
Epoch 10/30
0.00023264429182745516
0.00024739952641539276
0.00021929822105448693
0.0002488

[0.00025159408687613904,
 0.0002567078627180308,
 0.0002347471017856151,
 0.0002623775217216462,
 0.0002566072216723114,
 0.00025404829648323357,
 0.00025616673519834876,
 0.00027566897915676236,
 0.00022701302077621222,
 0.00026205735048279166,
 0.00026155426166951656,
 0.0002506707387510687,
 0.0002485751756466925,
 0.000243923015659675,
 0.00021555279090534896,
 0.00024902125005610287,
 0.000250117271207273,
 0.00026644422905519605,
 0.00021542387548834085,
 0.00020635980763472617,
 0.0002318520419066772,
 0.00024445666349492967,
 0.0002500811533536762,
 0.0002262728230562061,
 0.00022470254043582827,
 0.0002388863795204088,
 0.00023566283925902098,
 0.0002182702737627551,
 0.00025422341423109174,
 0.0002464283024892211,
 0.00022828884539194405,
 0.00024847075110301375,
 0.00020504763233475387,
 0.00023537101515103132,
 0.0001939756766660139,
 0.00023538354435004294,
 0.00023264429182745516,
 0.00024739952641539276,
 0.00021929822105448693,
 0.0002488992176949978,
 0.000257138919550

In [65]:
test = torch.from_numpy(tfidf.values).type(torch.FloatTensor).cuda()
test = test.unsqueeze(0)

In [66]:
pred = encoder(test)

In [67]:
pred = pred.cpu()
pred = pred.data.numpy()

In [68]:
pred.shape

(1, 26744, 100)

In [69]:
pred = pred.reshape((pred.shape[1], -1))

In [70]:
pred.shape

(26744, 100)

In [71]:
from sklearn.metrics.pairwise import cosine_similarity

In [76]:
with open('../data/autoencoder_embeddings.pkl', 'wb') as fh:
    pickle.dump(pred, fh)